## **Notebook EC#08**
## Encoder-Decoder LSTM for Natural Language Processing.

#### Este notebook está baseado no material presente em Brownlee, J. “Long Short-Term Memory Networks With Python – Develop Sequence Prediction Models With Deep Learning”, Machine Learning Mastery, 2017, mais especificamente no [Chapter 9].

**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Túlio Queiroz de Paula


In [24]:
import warnings
warnings.filterwarnings('ignore')

from random import seed
from random import randint
from numpy import array
from numpy import argmax

In [25]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X,y = list(), list()
    for i in range(n_examples):
        in_pattern=[randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X,y

In [26]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

[[3, 10]] [13]


In [27]:
from math import ceil
from math import log10

In [28]:
def to_string(X,y,n_numbers,largest):
    max_length = n_numbers*ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    maxlength = ceil(log10(n_numbers*(largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' 'for _ in range(maxlength-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [29]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

In [30]:
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

X,y = to_string(X,y,n_numbers,largest)
print(X,y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [31]:
alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

In [32]:
def integer_encode(X,y,alphabet):
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [33]:
X,y = integer_encode(X,y,alphabet)

In [34]:
print(X,y)

[[11, 3, 10, 1, 0]] [[1, 3]]


In [35]:
def one_hot_encode(X,y,max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)

    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [36]:
X,y = one_hot_encode(X,y,len(alphabet))

In [37]:
print(X,y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [38]:
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y

<font color="green">
Atividade (a) <br>
Explique, passo-a-passo, todas as etapas envolvidas na concepção dos dados de treinamento.
</font>

**Resposta:**<br>
As primeiras funções servem para gerar uma lista de números pseudo-aleatórios e computar a soma dos mesmos em uma outra lista. Em seguida, a função <i>to_string()</i> irá unir os dígitos com o símbolo '+' e converter tudo para string, considerando o tamanho necessário para representar <i>largest</i> (através de <i>ceil(log10(largest+1))</i>) e padronizando todas as strings para o mesmo tamanho (adicionando espaços em branco quando necessário). Após esse processo, temos vetores de strings com a operação de adição associados a vetores de strings com o resultado dessa soma. Prosseguindo, precisa-se definir o universo de símbolos que estão disponíveis para serem utilizados. Temos, portanto, <i>alphabet</i> como alfabeto que contempla todos os símbolos em questão. Por fim, cada símbolo deve ser mapeado em um vetor one-hot-encoded que representará a informação do mesmo - vetor de zeros com '1' na posição correspondente ao símbolo em <i>alphabet</i>

In [39]:
def invert(seq,alphabet):
    int_to_char = dict((i,c) for i,c in enumerate(alphabet))
    strings  = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [40]:
n_terms = 3
largest = 10
alphabet = [str(x) for x in range(10)] + ['+', ' ']

In [41]:
n_chars = len(alphabet)
n_in_seq_length = n_terms*ceil(log10(largest+1)) +n_terms-1
n_out_seq_length = ceil(log10(n_terms*(largest+1)))

In [42]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dense

In [43]:
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(n_chars,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 75)             │        26,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_1 (RepeatVector)  │ (None, 2, 75)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 2, 50)          │        25,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 2, 12)          │           612 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,212 (203.95 KB)

 Trainable params: 52,212 (203.95 KB)

 Non-trainable params: 0 (0.00 B)

None


In [44]:
X,y = generate_data(75000,n_terms,largest,alphabet)
model.fit(X,y,epochs=1,batch_size=10)

7500/7500 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.7330 - loss: 0.8193


In [45]:
X,y = generate_data(100,n_terms,largest,alphabet)
loss,acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.034685, Accuracy: 99.500000


In [46]:
for _ in range(10):
    X,y = generate_data(1,n_terms,largest,alphabet)
    yhat = model.predict(X,verbose=0)
    in_seq = invert(X[0],alphabet)
    out_seq = invert(y[0],alphabet)
    predicted = invert(yhat[0],alphabet)
    print('%s = %s (expect %s)' %(in_seq,predicted,out_seq))

  5+2+10 = 17 (expect 17)
   6+4+5 = 15 (expect 15)
   9+2+2 = 13 (expect 13)
  4+10+7 = 21 (expect 21)
 10+10+2 = 22 (expect 22)
   3+8+4 = 15 (expect 15)
   1+8+1 = 10 (expect 10)
   3+5+9 = 17 (expect 17)
   6+1+5 = 12 (expect 12)
   8+2+2 = 12 (expect 12)


<font color="green">
Atividade (b) <br>
Explique como uma calculadora simples pode operar baseada no conceito de tradução de frases, ou seja, sem realizar operações algébricas.
</font>

**Resposta:** <br>
Como nesse notebook, em um cenário que temos um conjunto finito (e pequeno) de números limitados por um máximo e apenas um símbolo de operação ('+'), é razoavelmente simples implementar uma calculadora baseada em regras, pois basta seguir o procedimento feito: cria-se um código, ou seja, uma lista de símbolos que transmitirão as informações - no exemplo desse notebook, símbolos de 0 a 9, ' ' e +. <br>Quando temos como entrada os vetores one-hot-encoded [11] (veja como [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), [3], [10], [1], [0] que representam " 3+10" e que, por sua vez, representa a operação 3+10, a "calculadora" foi treinada para associar tais vetores aos vetores one-hot-encoded [1], [3], que por sua vez representam "13", o qual equivale ao resultado da adição desejada. <br> Com um conjunto de códigos (um alfabeto suficientemente abrangente) e dados em quantidade adequada para treinamento, é possível expandir esse procedimento para outras operações, bastando treinar o algoritmo para reconhecer os novos símbolos ('*', '/', '-', etc.) juntamente com os demais símbolos já existentes. O "livro de regras" do algoritmo será ampliado para cobrir os novos casos e os "cálculos" serão realizados associando os vetores que representam os valores de entrada com os vetores - mapeados durante o treinamento - que representam a resposta.

<font color="green">
Atividade (c) <br>
Procure associar este experimento com o cenário da sala chinesa de Searle.
</font>

**Resposta:**<br>
A associação mais relevante advém do fato de que tanto o operador da sala chinesa quanto o algoritmo não compreendem o significado do que estão fazendo. O operador **não** está traduzindo as sentenças que recebe, pois ao traduzir uma frase, pressupõe-se compreensão - traduz-se frases justamente com o intuito de compreendê-las. O operador recebeu um código, um livro de regras (em seu idioma, ou seja, o operador compreende **as regras**) que dita apenas quais elementos do código desconhecido devem ser associados para responder a sentença recebida de forma adequada. O experimento desse notebook, por sua vez, não resolve operações matemáticas implementando de fato as operações. O algoritmo recebe regras (conjunto de dados de treinamento) que indicarão a resposta adequada quando tais símbolos de entrada aparecerem novamente. Pode-se fazer, por exemplo, uma operação "a + / = 8", basta que exista um "alfabeto" que cubra todos esses símbolos e dados de treinamento que informem ao algoritmo: <br> <br>
<i> "Ao receber os vetores que representam os símbolos "a+/", responda com o vetor que representa o símbolo "8"</i>  <br> <br>Assim, fica evidente que não há uma compreensão do que está sendo feito - não há nenhum sentido matemático a expressão "a + /" e em uma calculadora tradicional resultaria em erro -, apenas uma associação de símbolos dado um conjunto de regras (dados de treinamento).